<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>pg_cron</title>
    <style>
        .title {
            color: lightblue;
            font-size: 2.5em;
        }
        .description {
            font-size: 1.2em;
        }
        .description-title {
            color: orange;
            font-weight: bold;
        }
        .version-title {
            color: orange;
            font-weight: bold;
            font-size: 1.2em;
        }
        .version-number {
            color: white;
            font-size: 1.2em;
        }
    </style>
</head>
<body>
    <div class="title">DB_Ops BI as a Code</div>
    <div class="description">
        <span class="description-title">Description:</span> Testing how to generate data (with parameters) and UI from a single config element
    </div>
    <div>
        <span class="version-title">Version:</span> <span class="version-number">1.0</span>
    </div>
</body>
</html>


# Overview
- The config file `db_ops_config.yaml` contains multiple "operation" objects. Each objects contains the metadata to run a query (sql, parameters) and the default visualization.

# The `DB_Ops` object: 
- list the exsiting operations
- Get the SQL of an operation, with or without parameters    
- Generate a Data Frame by providing a connection string. 
    - Managing the connections string is NOT part of the db_ops library. It is a secret, should be managed externally. 
- Generate the default visualization

In [3]:
## Step 1 - Get the Operation configuration
import importlib
import db_ops
importlib.reload(db_ops)
from db_ops import DB_Ops




try: 
    # Create an instance of DB_Ops
    db_ops = DB_Ops()
    # Print every existing operation
    # print(db_ops.list_operations());

    # Example 2: Using custom parameter
    params = {"duration_interval": "7 days"}
    sql2 = db_ops.generate_sql_command("show_pg_cron_jobs_history_overview", params)
    print(f"SQL for show_pg_cron_jobs_history_overview (custom interval):\n{sql2}")

    # Example 1: Using default parameter
    sql1 = db_ops.generate_sql_command("show_pg_cron_jobs_history_overview")
    print(f"SQL for show_pg_cron_jobs_history_overview (using default):\n{sql1}")
    print()

   
except ValueError as e:
    print(f"Error: {str(e)}")



SQL for show_pg_cron_jobs_history_overview (custom interval):
SELECT
  j.jobid,
  j.jobname,
  COUNT(jrd.*) AS total_runs,
  SUM(CASE WHEN jrd.status = 'succeeded' THEN 1 ELSE 0 END) AS success_count,
  SUM(CASE WHEN jrd.status = 'failed' THEN 1 ELSE 0 END) AS failure_count,
  ROUND((SUM(CASE WHEN jrd.status = 'succeeded' THEN 1 ELSE 0 END)::DECIMAL / COUNT(jrd.*)) * 100, 2) AS success_percentage,
  ROUND((SUM(CASE WHEN jrd.status = 'failed' THEN 1 ELSE 0 END)::DECIMAL / COUNT(jrd.*)) * 100, 2) AS failure_percentage
FROM
    cron.job j
JOIN
    cron.job_run_details jrd ON j.jobid = jrd.jobid
WHERE
    jrd.start_time >= CURRENT_DATE - INTERVAL '7 days'
GROUP BY
    j.jobid, j.jobname
ORDER BY
    j.jobid;
SQL for show_pg_cron_jobs_history_overview (using default):
SELECT
  j.jobid,
  j.jobname,
  COUNT(jrd.*) AS total_runs,
  SUM(CASE WHEN jrd.status = 'succeeded' THEN 1 ELSE 0 END) AS success_count,
  SUM(CASE WHEN jrd.status = 'failed' THEN 1 ELSE 0 END) AS failure_count,
  ROUND((SUM

### Generate a Data Frame

In [7]:
# Get a Connection String
import configparser

config = configparser.ConfigParser()
config.read_file(open(r'../db_ops.cfg'))

con_str = config.get('con_str', 'PG_TEMBO_DEV1') 

## Set a connection string. Notice! this is just a string. The connection is not open yet. 
db_ops.active_con_str = con_str 
sql1 = db_ops.generate_sql_command("show_pg_cron_jobs_history_overview")
df = db_ops.execute_sql_query(con_str, sql1)



,jobid,jobname,total_runs,success_count,failure_count,success_percentage,failure_percentage
0,3,Find a random student,44366,44366,0,100.0,0.0
1,8,DB Activity B,44366,44366,0,100.0,0.0
2,9,test_booking_1,44366,44366,0,100.0,0.0
3,12,test_booking_select_2,44366,44366,0,100.0,0.0
4,13,test_3,44366,0,44366,0.0,100.0
5,14,insert_random_students,44366,44366,0,100.0,0.0
6,15,select_1000_students,8872,8872,0,100.0,0.0
7,16,insert_pg_stat_database_history,44366,44366,0,100.0,0.0
8,17,job_update_5_rows_in_bookings.bookings,8872,8872,0,100.0,0.0
9,18,job_select_10k_rows_from_bookings.flights,2958,2958,0,100.0,0.0


## Use the default Visualization